In [ ]:
# import packages

import pandas as pd
import os
import numpy as np

In [ ]:
# define data and files path

data_path = "C:/Users/Ander Merketegi/Desktop/GitHub/Data/PatientReportedOutcome/"
files_path = data_path + "Files/"

In [ ]:
# print number of articles in each folder

n_files = 0
for folder in os.listdir(files_path):
    n = len([file for file in os.listdir(files_path + folder) if file.endswith('.pdf')])
    n_files += n
    print(f"Number of files in folder {folder}: {n}")
print(f"\nTotal number of files: {n_files}")

Number of files in folder 2013_14: 1111
Number of files in folder 2015: 723
Number of files in folder 2016: 721

Total number of files: 2555


We can see in the output of the cell above that we have a **total of 2555 files available.** In the following cells, we are going to read and analyze the excel files in which information about the articles is stored.

As an example, we will read the excel file corresponding to 2013-14 articles.

In [ ]:
# read excel file and save as a dataframe

df_2013 = pd.read_excel(data_path + "bd_2013_2014.xlsx").replace(np.nan, '', regex = True) #replace NaN values with an empty string
display(df_2013.head())

print(f"\nTotal number of files analyzed in 2013-14: {len(df_2013)}")
print(f"Number of files that passed title-abstract filter: {len(df_2013.loc[df_2013['filtro_titol'] == 1])}")
print(f"Number of files that passed instrument filter: {len(df_2013.loc[df_2013['filtro_instrumento'] == 1])}")

,pmid,refid,filtro_titol,filtro_instrumento,Instrumentos_BiblioPRO_1,BiblioPRO_SIGLAS_1,Instrumentos_BiblioPRO_2,BiblioPRO_SIGLAS_2,Instrumentos_BiblioPRO_3,BiblioPRO_SIGLAS_3,...,NUEVO_SIGLAS_1,Instrumento_NUEVO_ESP_2,Instrumento_NUEVO_ENG_2,NUEVO_SIGLAS_2,Instrumento_NUEVO_ESP_3,Instrumento_NUEVO_ENG_3,NUEVO_SIGLAS_3,Instrumento_NUEVO_ESP_4,Instrumento_NUEVO_ENG_4,NUEVO_SIGLAS_4
0,PM:21245050,2427,1,1,Depression in the Medically Ill questionnaire ...,DMI-18,Depression in the Medically Ill questionnaire ...,DMI-10,Beck Depression Inventory-II,BDI-II,...,BDI-PC,,,,,,,,,
1,PM:21435750,2426,1,0,,,,,,,...,,,,,,,,,,
2,PM:21467092,2425,1,1,Cognitive Emotion Regulation Questionnaire,CERQ,Inventario de Depresión de Beck,BDI,Versión española del State-Trait Anxiety Inven...,STAI,...,,,,,,,,,,
3,PM:21477827,2424,0,0,,,,,,,...,,,,,,,,,,
4,PM:21529862,2423,0,0,,,,,,,...,,,,,,,,,,



Total number of files analyzed in 2013-14: 4230
Number of files that passed title-abstract filter: 1124
Number of files that passed instrument filter: 898


In [ ]:
# print number of unique documents that passed titol filter

print(f"Number of unique files that passed titol filter (titol == 1) : {len(set(df_2013.loc[df_2013['filtro_titol'] == 1]['pmid']))}")
print(f"Number of unique files that passed intrument filter (filtro_instrumento == 1) : {len(set(df_2013.loc[df_2013['filtro_instrumento'] == 1]['pmid']))}")

Number of unique files that passed titol filter (titol == 1) : 1112
Number of unique files that passed intrument filter (filtro_instrumento == 1) : 887


We can observe here that the number of documents we have is consistent with the number of unique documents that went through the title/abstract filter (except one document). Thus, we have **1111 articles from which 887 contain a PROM** within.

In the next cells we are going to create a vocabulary with all the Patient Reported Outcome Questionnaires.

## Create PROM vocabulary

In [ ]:
proms = {}
index_col = 0
index_dict = 0

df_proms = df_2013.iloc[:, 4:]
n_cols = len(df_proms.columns)

for x in df_proms.itertuples(index = False):
    while index_col < n_cols - 1:
        if (x[index_col], [index_col + 1]) not in list(proms.values()) and not (x[index_col] == '' and x[index_col + 1] == ''):
            proms[index_dict] = (x[index_col], x[index_col + 1])
            index_dict += 1
        index_col += 2
    index_col = 0